In [67]:
import pandas as pd

train_and_pretest_data = pd.read_csv('data/detect_ai.csv')
test_data = pd.read_csv('data/daigt_v4.csv')

train_and_pretest_data = train_and_pretest_data.drop_duplicates(subset='text')
test_data = test_data[~test_data['text'].isin(train_and_pretest_data['text'])]

print(f"Train and pretest data size: {len(train_and_pretest_data)}")
print(f"Test data size: {len(test_data)}")


Train and pretest data size: 158294
Test data size: 40202


In [68]:
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import pandas as pd

def preprocess_data(data):
    tokens = word_tokenize(data.lower())
    return " ".join(tokens)

train_pretest_tokenized_df = pd.DataFrame(columns=['tokens', 'label'])
test_tokenized_df = pd.DataFrame(columns=['tokens', 'label'])

for index, row in train_and_pretest_data.iterrows():
    train_pretest_tokenized_df = pd.concat([train_pretest_tokenized_df, pd.DataFrame({'tokens': preprocess_data(row['text']), 'label': row['generated']}, index=[0])], ignore_index=True)

for index, row in test_data.iterrows():
    test_tokenized_df = pd.concat([test_tokenized_df, pd.DataFrame({'tokens': preprocess_data(row['text']), 'label': row['label']}, index=[0])], ignore_index=True)

train_data, pretest_data = train_test_split(train_pretest_tokenized_df, stratify=train_pretest_tokenized_df.label, test_size=0.2)

train_data.to_csv("data/baseline_processed_train_data.csv")
pretest_data.to_csv("data/baseline_processed_pretest_data.csv")
test_tokenized_df.to_csv("data/baseline_processed_test_data.csv")


In [69]:
train_data = pd.read_csv("data/baseline_processed_train_data.csv")
pretest_data = pd.read_csv("data/baseline_processed_pretest_data.csv")
test_data = pd.read_csv("data/baseline_processed_test_data.csv")

print(f"Number of entries in the training data: {train_data.shape[0]}")
print(f"Number of entries in the test data: {pretest_data.shape[0]}")

data_label_0_proportion = train_and_pretest_data[train_and_pretest_data.generated == 0].shape[0] / train_and_pretest_data.shape[0]
data_label_1_proportion = train_and_pretest_data[train_and_pretest_data.generated == 1].shape[0] / train_and_pretest_data.shape[0]

train_data_label_0_propotion = train_data[train_data.label == 0].shape[0] / train_data.shape[0]
train_data_label_1_propotion = train_data[train_data.label == 1].shape[0] / train_data.shape[0]

pretest_data_label_0_propotion = pretest_data[pretest_data.label == 0].shape[0] / pretest_data.shape[0]
pretest_data_label_1_propotion = pretest_data[pretest_data.label == 1].shape[0] / pretest_data.shape[0]

print("\n\nProportion of the data:")
print(f"\n{'':<20s} {'Data overall':<20s} {'Train data':<20s} {'Test data':<20s}")
print(f"{'Human written':<20s} {data_label_0_proportion:<20.4f} {train_data_label_0_propotion:<20.4f} {pretest_data_label_0_propotion:<20.4f}")
print(f"{'LLM generated':<20s} {data_label_1_proportion:<20.4f} {train_data_label_1_propotion:<20.4f} {pretest_data_label_1_propotion:<20.4f}")


Number of entries in the training data: 126635
Number of entries in the test data: 31659


Proportion of the data:

                     Data overall         Train data           Test data           
Human written        0.2288               0.2288               0.2288              
LLM generated        0.7712               0.7712               0.7712              


In [70]:
def get_features(text):
    """
    A simple feature extractor, based on Kochmar, 2022, p. 171

    :param text: a string
    :return: a dictionary of features
    """
    features = {}
    words = text.split(' ')
    for word in words:
        features[word.lower()] = True
    return features

train_features = [(get_features(row['tokens']), row['label']) for _, row in train_data.iterrows()]
pretest_features = [(get_features(row['tokens']), row['label']) for _, row in pretest_data.iterrows()]
test_features = [(get_features(row['tokens']), row['label']) for _, row in test_data.iterrows()]

print(f"Number of entries in the features of the training data: {len(train_features)}")
print(f"Number of entries in the features of the test data: {len(pretest_features)}")

print(train_features[0])




Number of entries in the features of the training data: 126635
Number of entries in the features of the test data: 31659
({'effects': True, 'of': True, 'adding': True, 'an': True, 'additional': True, '1.5': True, 'hours': True, 'to': True, 'the': True, 'school': True, 'day': True, 'on': True, 'student': True, 'learning': True, 'and': True, 'grades': True, '.': True, 'it': True, 'is': True, 'widely': True, 'accepted': True, 'that': True, 'students': True, 'need': True, 'more': True, 'time': True, 'learn': True, 'succeed': True, 'academically': True, 'in': True, 'recent': True, 'years': True, ',': True, 'there': True, 'has': True, 'been': True, 'a': True, 'growing': True, 'trend': True, 'towards': True, 'extra': True, 'with': True, 'some': True, 'schools': True, 'even': True, 'extending': True, 'by': True, 'up': True, 'purpose': True, 'this': True, 'essay': True, 'explore': True, 'evidence': True, 'supporting': True, 'idea': True, 'providing': True, 'for': True, 'helps': True, 'examine':

In [71]:
from nltk import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(train_features)

In [72]:
from sklearn.metrics import f1_score

labels_train = [label for _, label in train_features]
labels_pretest = [label for _, label in pretest_features]
labels_test = [label for _, label in test_features]
predicted_labels_train = [classifier.classify(featureset) for featureset, _ in train_features]
predicted_labels_pretest = [classifier.classify(featureset) for featureset, _ in pretest_features]
predicted_labels_test = [classifier.classify(featureset) for featureset, _ in test_features]
print(f"F1 score on the training data: {f1_score(labels_train, predicted_labels_train)}")
print(f"F1 score on the pretest data: {f1_score(labels_pretest, predicted_labels_pretest)}")
print(f"F1 score on the test data: {f1_score(labels_test, predicted_labels_test)}")

F1 score on the training data: 0.7442137690024908
F1 score on the pretest data: 0.7263287885279367
F1 score on the test data: 0.9605975154332749


In [36]:
test_v2_data = pd.read_csv('data/ai_vs_human.csv')
train_and_pretest_data = pd.read_csv('data/detect_ai.csv')
test_data = pd.read_csv('data/daigt_v4.csv')

test_v2_data = test_v2_data[~test_v2_data['text'].isin(train_and_pretest_data['text'])]
test_v2_data = test_v2_data[~test_v2_data['text'].isin(test_data['text'])]

print(len(test_v2_data))

test_v2_tokenized_df = pd.DataFrame(columns=['tokens', 'label'])

for index, row in test_v2_data.iterrows():
    test_v2_tokenized_df = pd.concat([test_v2_tokenized_df, pd.DataFrame({'tokens': preprocess_data(row['text']), 'label': row['generated']}, index=[0])], ignore_index=True)

test_v2_features = [(get_features(row['tokens']), row['label']) for _, row in test_v2_tokenized_df.iterrows()]

labels_test_v2 = [label for _, label in test_v2_features]
predicted_labels_test_v2 = [classifier.classify(featureset) for featureset, _ in test_v2_features]
print(f"F1 score on the test data v2: {f1_score(labels_test_v2, predicted_labels_test_v2)}")

426765


/var/folders/9w/2pytp4_s6bj5_ff4zz3vl8ph0000gn/T/ipykernel_16176/592256311.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_v2_tokenized_df = pd.concat([test_v2_tokenized_df, pd.DataFrame({'tokens': preprocess_data(row['text']), 'label': row['generated']}, index=[0])], ignore_index=True)


F1 score on the test data v2: 0.8521676368254458
